In [3]:
from qchart import start_listener, listener_is_running
from qchart.client import DataSender
from datetime import datetime
import copy
import time
import numpy as np

def get_new_id():
    # use this to tag datasets
    t = datetime.now().strftime('%Y-%M-%d_%H-%M-%S-%f')
    return f'test_nik_{t}'

In [4]:
from functools import partial

def sine_model(x, freq: float, phase: float, noise: float=1):
    """
    Returns a generator sampling a sine wave. The sine wave has an 
    amplitude of 1 and some noise
    """

    rad_phase = np.pi*phase/180.0
    model = np.sin(2*np.pi*x + rad_phase)
    noise = np.random.rand(*model.shape)*noise
    return np.array(model + noise)

def gauss_model(x, y, x0: float, y0: float, sigma: float, noise: float=0.0005):
    """
    Returns a generator sampling a gaussian. The gaussian is
    normalised such that its maximal value is simply 1
    """
    model = np.exp(-((x0-x)**2+(y0-y)**2)/2/sigma**2)*np.exp(2*sigma**2)
    noise = np.random.rand(*model.shape)*noise
    return (model + noise)

In [5]:
if not listener_is_running():
    start_listener()

starting qchart listener at c:\users\nikha\repos\qchart\listener_start.py ...
qchart listener successfully started.


### 1D data in one shot

In [ ]:
# template for sending data through DataSender
data_structure = {
    'x_param': {
        'values': [],
        'unit': 'A'
    },
    'y_param': {
        'values': [],
        'unit': '',
        'axes': ['x_param']
    }
}

# getter function
get_sin1 = partial(sine_model, freq=10, phase = 9, noise=0.1)

x_vals = np.linspace(3,4,100)
y_vals = get_sin1(x_vals)

sndr = DataSender(get_new_id())

data = copy.deepcopy(data_structure)
data['x_param']['values'] = x_vals
data['y_param']['values'] = y_vals

sndr.send_data(data)

### 1D data in 1 shot with 2 outputs

In [ ]:
# template for sending data through DataSender
data_structure = {
    'x_param': {
        'values': [],
        'unit': 'mA'
    },
    'y_param': {
        'values': [],
        'unit': 'arb',
        'axes': ['x_param']
    },
    'y_param_2': {
        'values': [],
        'unit': 'arb',
        'axes': ['x_param']
    }
}

# getter function
get_sin1 = partial(sine_model, freq=10, phase = 9, noise=0.1)
get_sin2 = partial(sine_model, freq=12, phase = 99, noise=0.02)

x_vals = np.linspace(3,4,100)

sndr = DataSender(get_new_id())

data = copy.deepcopy(data_structure)
data['x_param']['values'] = x_vals
data['y_param']['values'] = get_sin1(x_vals)
data['y_param_2']['values'] = get_sin2(x_vals)

sndr.send_data(data)

### Stream 1D data

In [17]:
# template for sending data through DataSender
data_structure = {
    'x_param': {
        'values': [],
        'unit': 'mA'
    },
    'y_param': {
        'values': [],
        'unit': 'arb',
        'axes': ['x_param']
    },
    'y_param_2': {
        'values': [],
        'unit': 'arb',
        'axes': ['x_param']
    }
}

# getter function
get_sin1 = partial(sine_model, freq=10, phase = 9, noise=0.1)
get_sin2 = partial(sine_model, freq=12, phase = 99, noise=0.02)

x_vals = np.linspace(0,100,1000).reshape(20,-1)

sndr = DataSender(get_new_id())

for x in x_vals:
    data = copy.deepcopy(data_structure)
    data['x_param']['values'] = x
    data['y_param']['values'] = get_sin1(x)
    data['y_param_2']['values'] = get_sin2(x)

    time.sleep(1.0) # artifical delay to pretent like we're collecting data

    sndr.send_data(data)

### 2D data in a few shots

In [6]:
# template for sending data through DataSender
data_structure = {
    'x_param': {
        'values': [],
        'unit': 'mA'
    },
    'y_param': {
        'values': [],
        'unit': 'arb',
    },
    'z_param': {
        'values': [],
        'unit': 'arb',
        'axes': ['x_param','y_param']
    }
}

# getter function
get_gauss1 = partial(gauss_model, x0=0, y0=0, sigma=0.2, noise=0.08)

x_vals = np.linspace(-0.5,1.5,40)
y_vals = np.linspace(-1,1,40)

grid = np.stack(np.meshgrid(y_vals,x_vals)).reshape(2,-1).T
chunks = 19
points_per_send = grid.shape[0] // chunks


sndr = DataSender(get_new_id())

for ii in range(chunks):
    
    start = ii*points_per_send
    if ii + 1 == chunks:
        end = grid.shape[0]
    else:
        end = (ii + 1)*points_per_send

    data = copy.deepcopy(data_structure)
    data['x_param']['values'] = grid[start:end,1]
    data['y_param']['values'] = grid[start:end,0]
    data['z_param']['values'] = get_gauss1(grid[start:end,1], grid[start:end,0])

    sndr.send_data(data)

    time.sleep(0.5)

### 2D point by point

In [7]:
# template for sending data through DataSender
data_structure = {
    'x_param': {
        'values': [],
        'unit': 'mA'
    },
    'y_param': {
        'values': [],
        'unit': 'arb',
    },
    'z_param': {
        'values': [],
        'unit': 'arb',
        'axes': ['x_param','y_param']
    }
}

# getter function
get_gauss1 = partial(gauss_model, x0=0, y0=0, sigma=0.2, noise=0.08)

x_vals = np.linspace(-0.5,1.5,40)
y_vals = np.linspace(-1,1,40)

sndr = DataSender(get_new_id())

for x in x_vals:
    for y in y_vals:

        data = copy.deepcopy(data_structure)
        data['x_param']['values'] = x
        data['y_param']['values'] = y
        data['z_param']['values'] = get_gauss1(x,y)

        sndr.send_data(data)

        time.sleep(0.02)

KeyboardInterrupt: 

In [11]:
a = np.array(data['z_param']['values'], dtype=float)

In [14]:
v = 0.1

In [19]:
np.array(v, dtype=float).reshape((-1,))

array([0.1])